# melody basic rnn
## Douglas Eck
This notebook shows you how to generate new performed compositions from a trained Performance RNN model. You'll see how to download a bundle containing a pre-trained model, instantiate and initialize the model and generate new polyphonic performances. The notebook also shows some hyperparameters useful for controlling generation, such as ``temperature``.

To hear the performances it is useful to have fluidsynth and pyfluidsynth installed. One way to install fluidsynth on OS X is via homebrew: ``brew install fluidsynth``.  One way to install pyfluidsynth is via pip: ``pip install pyfluidsynth``.

In [ ]:
import os
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import magenta.music as mm

# Constants.
BUNDLE_DIR = '/home/jovyan/models/'
MODEL_NAME = 'lookback_rnn' # one of 'basic_rnn', 'attention_rnn', 'lookback_rnn'
BUNDLE_NAME = 'lookback_rnn.mag'

In [ ]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))

In [ ]:
generator_map = melody_rnn_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)

In [ ]:
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = .9  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

In [ ]:
qpm = 160

generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = .9  # Higher is more random; 1.0 is default. 
generator_options.args['qpm'].float_value = qpm

seconds_per_step = 120.0 / qpm / generator.steps_per_quarter
total_seconds = 64 * seconds_per_step
last_end_time = 0

generate_section = generator_options.generate_sections.add(start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

In [ ]:
generator_options.args['temperature'].float_value = 1.1  # Higher is more random; 1.0 is default. 
#sequence = generator.generate(music_pb2.NoteSequence(), generator_options)
#mm.plot_sequence(sequence)
#generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

In [ ]:
from magenta.music.midi_io import sequence_proto_to_pretty_midi, sequence_proto_to_midi_file

In [ ]:
sequences = []
for i in range(16):
    temperature = [.6,.8,.9,1.8,.6,.8,.9,1.8][i % 8]
    generator_options.args['temperature'].float_value = temperature
    sequence = generator.generate(music_pb2.NoteSequence(), generator_options)
    sequences.append(sequence)
    pm = sequence_proto_to_pretty_midi(sequence)

In [ ]:
from magenta.music.sequences_lib import concatenate_sequences, quantize_note_sequence

In [ ]:
big_seq = concatenate_sequences(sequences)

In [ ]:
mm.play_sequence(big_seq, mm.midi_synth.fluidsynth)

In [ ]:
sequence_proto_to_midi_file(big_seq, 'lookback_rnn.mid')

In [ ]:
with open('lookback_rnn.sequence','wb') as f:
    f.write(big_seq.SerializeToString())

In [ ]:
from magenta.music.chord_inference import infer_chords_for_sequence

In [ ]:
infer_chords_for_sequence(big_seq)

In [ ]:
infer_chords_for_sequence(quantize_note_sequence(big_seq, qpm))